In [ ]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import torch
import torchvision
import torchvision.transforms as T
from collections import defaultdict, deque
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator
import ast
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SequentialSampler
import torchvision.transforms as transforms
import cv2
import os,sys,matplotlib,re
from PIL import Image
from skimage import exposure
import matplotlib.pyplot as plt
import matplotlib.image as immg
from torchvision.io import read_image
import matplotlib
from torchvision.utils import draw_bounding_boxes
import math

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

In [ ]:
bbox_path = '/kaggle/input/tabulard/Train.csv'
image_folder_path = '/kaggle/input/imagest/train/train'

In [ ]:
bbox_path

In [ ]:
data_bbox = pd.read_csv(bbox_path).rename(columns={ 'Xmin': 'bbox_x1', 'Xmax': 'bbox_x2', 'Ymin': 'bbox_y1', 'Ymax': 'bbox_y2'})

data_bbox['img_path']=0

for im in os.listdir(image_folder_path):
    
    image_id = int(im.split('.')[0].split('_')[1])
    data_bbox.loc[data_bbox['Image_ID'] == image_id, 'img_path'] = im
data_bbox

In [ ]:
data_bbox.ToothClass.unique()

In [ ]:
data_bbox.info()

In [ ]:
data_bbox['bbox_x1'][0]

In [ ]:
i=6
img = read_image(image_folder_path +'/' +data_bbox['img_path'][i])
bbox = [data_bbox['bbox_x1'][i],data_bbox['bbox_y1'][i],data_bbox['bbox_x2'][i],data_bbox['bbox_y2'][i]]
bbox = torch.tensor(bbox, dtype=torch.int)
bbox = bbox.unsqueeze(0)
print(bbox)
img=torchvision.utils.draw_bounding_boxes(img, bbox, width=3, colors=(255,255,0))
img = torchvision.transforms.ToPILImage()(img)
display(img)

In [ ]:
df = data_bbox.copy()

* In order to have correct implementation, classes should be from 1-numClasses (0 for background)

In [ ]:
# df['labels']=df['ToothClass'] - df['ToothClass'].min()+1
# Original labels
original_labels = df['ToothClass'].unique().tolist()

# Create the mapping dictionary
label_mapper = {original_label: new_label for new_label, original_label in enumerate(sorted(original_labels), start=1)}

# Apply the mapping to the DataFrame column
df['labels'] = df['ToothClass'].map(label_mapper)

len(df[['labels']].value_counts()), df[['labels']].value_counts()

In [ ]:
df.labels.min()

In [ ]:
df.labels

In [ ]:
len(df['img_path'].unique().tolist())

# Create data class:

In [ ]:
image_id

In [ ]:
cv2.imread('/kaggle/input/imagest/train/train/ID_000000.png',cv2.IMREAD_COLOR)

In [ ]:
df['img_path'].unique().tolist()[9]

In [ ]:
class CarDataset(object):
    def __init__(self, df, IMG_DIR, transforms=None):
        self.a = 0
        self.df = df
        self.img_dir = IMG_DIR
        self.image_ids = self.df['img_path'].unique().tolist()
        self.transforms = transforms
        
    def __len__(self):
        return len(self.image_ids)
        
    def __getitem__(self, idx):
        image_id = self.image_ids[idx]
#         print(self.img_dir+image_id)
        records = self.df[self.df['img_path'] == image_id]
        
        image = cv2.imread(self.img_dir+'//'+image_id,cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)

#         print(image)
        image /= 255.0
#         display(records)
        boxes = records[['bbox_x1', 'bbox_y1', 'bbox_x2', 'bbox_y2']].to_numpy()
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        labels =  records['labels'].to_numpy()#torch.ones((records.shape[0],), dtype=torch.int64)
        
        target = {}
        target['boxes'] = boxes
        target['labels'] = labels
        target['image_id'] = torch.tensor([idx])
        target['area'] = torch.as_tensor(area, dtype=torch.float32)
        target['iscrowd'] = torch.zeros((records.shape[0],), dtype=torch.int64)
    
        if self.transforms:
            sample = {
                'image': image,
                'bboxes': target['boxes'],
                'labels': labels
            }
            sample = self.transforms(**sample)
            image = sample['image']
            
            target['boxes'] = torch.stack(tuple(map(torch.tensor, zip(*sample['bboxes'])))).permute(1, 0)
        return image.clone().detach(), target, image_id

In [ ]:

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
# def get_train_transform():
#     return A.Compose([
#         A.Flip(0.5),
#         ToTensorV2()
#     ], bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})

def get_train_transform():
    return A.Compose([
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.Rotate(limit=90, p=1.0),
        A.ColorJitter( contrast=0.8, saturation=0.8),
        ToTensorV2()
    ], bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})


def get_valid_transform():
    return A.Compose([
        ToTensorV2()
    ], bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})

Let`s show how work our class dataset

In [ ]:
CD = CarDataset(df, image_folder_path, get_train_transform())


In [ ]:
# Function to visualize a single image and its bounding boxes
def visualize(image, target):
    fig, ax = plt.subplots(1, 1, figsize=(12, 8))
    image = image.permute(1, 2, 0).cpu().numpy()  # Convert to HWC format
    ax.imshow(image)
    
    boxes = target['boxes'].cpu().numpy()
    for box in boxes:
        x1, y1, x2, y2 = box
        rect = plt.Rectangle((x1, y1), x2 - x1, y2 - y1, fill=False, color='red', linewidth=2)
        ax.add_patch(rect)
    plt.show()

# Visualize a few samples
for i in range(5):
    image, target, image_id = CD[i]
    print(f"Image ID: {image_id}")
    visualize(image, target)

In [ ]:
CD[10]

In [ ]:
fig = plt.figure(figsize=(14, 10))
sample = CD[1]

# print(sample[1])
img_int = torch.tensor(sample[0] * 255, dtype=torch.uint8)
plt.imshow(draw_bounding_boxes(
    img_int, sample[1]['boxes'], width=4
).permute(1, 2, 0))

In [ ]:
import matplotlib.pyplot as plt

for i in range(5):
    # Extract relevant data
    sample = CD[i]

    image_tensor = sample[0]
    boxes = sample[1]['boxes']
    labels = sample[1]['labels']

    # Convert image tensor to uint8
    img_uint8 = image_tensor.permute(1, 2, 0).numpy() * 255
    img_uint8 = img_uint8.astype(np.uint8)

    # Plot the image
    plt.figure(figsize=(14, 10))
    plt.imshow(img_uint8)

    # Draw bounding boxes with labels
    for box, label in zip(boxes, labels):
        x_min, y_min, x_max, y_max = box
        plt.gca().add_patch(plt.Rectangle((x_min, y_min), x_max - x_min, y_max - y_min,
                                          linewidth=2, edgecolor='r', facecolor='none'))
        plt.text(x_min, y_min - 5, f'{label}', color='r', fontsize=12, fontweight='bold')

    plt.show()


# Split data and create dataloaders:

In [ ]:
image_ids = df['img_path'].unique()
valid_ids = image_ids[-665:]
train_ids = image_ids[:-665]
valid_df = df[df['img_path'].isin(valid_ids)]
train_df = df[df['img_path'].isin(train_ids)]
train_df.shape,valid_df.shape

In [ ]:
def collate_fn(batch):
    return tuple(zip(*batch))

train_dataset = CarDataset(train_df, image_folder_path, get_train_transform())
valid_dataset = CarDataset(valid_df, image_folder_path, get_valid_transform())

indices = torch.randperm(len(train_dataset)).tolist()
train_data_loader = DataLoader(
    train_dataset,
    batch_size=2,
    shuffle=True,
    num_workers=2,
    collate_fn=collate_fn
)


valid_data_loader = DataLoader(
    valid_dataset,
    batch_size=2,
    shuffle=True,
    num_workers=2,
    collate_fn=collate_fn
)

# Model:
I use fasterrcnn_resnet50_fpn pre-trained on COCO dataset

In [ ]:
num_classes = 32+1  # 1 class (car) + background

model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True, backbone_name='resnet101')
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

#### Faster-RFormer

In [ ]:
import torchvision
# from torchvision.models.detection import FasterRCNN
# from torchvision.models.detection.rpn import AnchorGenerator
# from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
# from torchvision.models.detection.backbone_utils import resnet_fpn_backbone

# # Define the backbone
# backbone = resnet_fpn_backbone('resnet101', pretrained=True)

# # Define the number of feature maps generated by the backbone
# num_anchors = 5  # Number of anchor scales (corresponding to 5 feature maps)

# # Define the anchor generator
# anchor_sizes = ((32,), (64,), (128,), (256,), (512,))
# aspect_ratios = ((0.5, 1.0, 2.0),) * num_anchors
# anchor_generator = AnchorGenerator(sizes=anchor_sizes,
#                                    aspect_ratios=aspect_ratios)

# # Define the ROI pooler
# roi_pooler = torchvision.ops.MultiScaleRoIAlign(featmap_names=['0', '1', '2', '3'],
#                                                 output_size=7,
#                                                 sampling_ratio=2)

# # Create the Faster R-CNN model
# model = FasterRCNN(backbone=backbone,
#                    num_classes=num_classes,
#                    rpn_anchor_generator=anchor_generator,
#                    box_roi_pool=roi_pooler)

# # Modify the box predictor to match the number of classes
# in_features = model.roi_heads.box_predictor.cls_score.in_features
# model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

In [ ]:
model.to(device)
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.001, momentum=0.9, weight_decay=0.0005)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

In [ ]:
# model

# Train our model:

In [ ]:
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import sys
from tqdm import tqdm
model.to(device)

# # Path to the saved model weights
# saved_weights_path = '/kaggle/input/weights27/pytorch/modelweights/1/epoch_27_best_weights.pth'

# # Load the model weights
# model.load_state_dict(torch.load(saved_weights_path))

def train_one_epoch(model, optimizer, loader, device, epoch):
    model.to(device)
    model.train()
    
    all_losses = []
    all_losses_dict = []
    
    for images, targets, _ in tqdm(loader):
        images = list(image.to(device) for image in images)
        targets = [{k: torch.tensor(v).to(device) for k, v in t.items()} for t in targets]
        
        loss_dict = model(images, targets)
#         if zinka==0:
#             zinka+=1
#             print(loss_dict)
#             print(type(loss_dict))
        losses = sum(loss for loss in loss_dict.values())
        loss_dict_append = {k: v.item() for k, v in loss_dict.items()}
        loss_value = losses.item()
        
        all_losses.append(loss_value)
        all_losses_dict.append(loss_dict_append)
        
        if not math.isfinite(loss_value):
            print(f"Loss is {loss_value}, stopping training")
            print(loss_dict)
            sys.exit(1)
        
        optimizer.zero_grad()
        losses.backward()
        optimizer.step()
        
    all_losses_dict = pd.DataFrame(all_losses_dict)
    avg_loss = np.mean(all_losses)
    print("Epoch {}, lr: {:.6f}, loss: {:.6f}, loss_classifier: {:.6f}, loss_box: {:.6f}, loss_rpn_box: {:.6f}, loss_object: {:.6f}".format(
        epoch, optimizer.param_groups[0]['lr'], avg_loss,
        all_losses_dict['loss_classifier'].mean(),
        all_losses_dict['loss_box_reg'].mean(),
        all_losses_dict['loss_rpn_box_reg'].mean(),
        all_losses_dict['loss_objectness'].mean()
    ))
    return avg_loss

global_loss_dict = None
def validate_one_epoch(model, loader, device):
    global global_loss_dict
    model.to(device)
#     model.eval()# never use val here because it make model(images, targets) make prediction instead on computing loss
    
    all_losses = []
    all_losses_dict = []
    
    with torch.no_grad():
        for images, targets, _ in tqdm(loader):
            images = list(image.to(device) for image in images)
            targets = [{k: torch.tensor(v).to(device) for k, v in t.items()} for t in targets]
            loss_dict = model(images, targets)
            global_loss_dict = loss_dict
#             print('loss_dict',loss_dict)
            
            if isinstance(loss_dict, dict):
                losses = sum(loss for loss in loss_dict.values())
                loss_dict_append = {k: v.item() for k, v in loss_dict.items()}
                loss_value = losses.item()
            else:
                print("Unexpected loss_dict format: ", type(loss_dict))
                loss_value = loss_dict.item()
                loss_dict_append = {"loss": loss_value}
            
            all_losses.append(loss_value)
            all_losses_dict.append(loss_dict_append)
    
    all_losses_dict = pd.DataFrame(all_losses_dict)
    avg_loss = np.mean(all_losses)
    print("Validation loss: {:.6f}, loss_classifier: {:.6f}, loss_box: {:.6f}, loss_rpn_box: {:.6f}, loss_object: {:.6f}".format(
        avg_loss,
        all_losses_dict.get('loss_classifier', pd.Series([0])).mean(),
        all_losses_dict.get('loss_box_reg', pd.Series([0])).mean(),
        all_losses_dict.get('loss_rpn_box_reg', pd.Series([0])).mean(),
        all_losses_dict.get('loss_objectness', pd.Series([0])).mean()
    ))
    return all_losses_dict.get('loss_box_reg', pd.Series([0])).mean()

def train_model(model, optimizer, train_loader, val_loader, num_epochs, device, patience):
    train_losses = []
    val_losses = []

    best_val_loss = float('inf')
    patience_counter = 0

    for epoch in range(num_epochs):
        print(f"Epoch {epoch + 1}/{num_epochs}")
        train_loss = train_one_epoch(model, optimizer, train_loader, device, epoch)
        val_loss = validate_one_epoch(model, val_loader, device)
        
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
            # Save model weights
            torch.save(model.state_dict(), f'epoch_{epoch}_best_weights.pth')
        else:
            patience_counter += 1

        if patience_counter >= patience:
            print(f"Early stopping at epoch {epoch + 1}")
            break
    
    plt.plot(range(1, len(train_losses) + 1), train_losses, label='Training Loss')
    plt.plot(range(1, len(val_losses) + 1), val_losses, label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Training and Validation Loss over Epochs')
    plt.legend()
    plt.show()

# Example usage:
num_epochs = 50
patience = 20
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# Assume train_loader and val_loader are your training and validation data loaders
# Example: train_loader = ...
#          val_loader = ...
train_model(model, optimizer, train_data_loader, valid_data_loader, num_epochs, device, patience)


#### Load weights

In [ ]:

# model.to(device)

# # Path to the saved model weights
# saved_weights_path = '/kaggle/input/weights27/pytorch/modelweights/1/epoch_27_best_weights.pth'

# # Load the model weights
# model.load_state_dict(torch.load(saved_weights_path))

# # Set the model to evaluation mode if you're using it for inference
# model.eval()

# Show resault:

In [ ]:
def apply_nms(orig_prediction, iou_thresh=0.2):
    

    keep = torchvision.ops.nms(orig_prediction['boxes'], orig_prediction['scores'], iou_thresh)
    
    final_prediction = orig_prediction
    final_prediction['boxes'] = final_prediction['boxes'][keep]
    final_prediction['scores'] = final_prediction['scores'][keep]
    final_prediction['labels'] = final_prediction['labels'][keep]
    
    return final_prediction

In [ ]:
img,target,_ = valid_dataset[1]
model.eval()
with torch.no_grad():
    prediction = model([img.to('cuda')])[0]
    
nms_prediction = apply_nms(prediction, iou_thresh=0.7)

In [ ]:
nms_prediction

In [ ]:
print("REAL BOXES")
fig = plt.figure(figsize=(14, 10))
sample = valid_dataset[10]
img_int = torch.tensor(sample[0] * 255, dtype=torch.uint8)
plt.imshow(draw_bounding_boxes(
    img_int, sample[1]['boxes'], width=4
).permute(1, 2, 0))

In [ ]:
print("PREDICTED BOXES")
fig = plt.figure(figsize=(14, 10))
sample = valid_dataset[109]
img_int = torch.tensor(sample[0] * 255, dtype=torch.uint8)
plt.imshow(draw_bounding_boxes(img_int,
    nms_prediction['boxes'][nms_prediction['scores'] > 0.5], width=4
).permute(1, 2, 0))

# Prediction

In [ ]:
test_df = pd.read_csv('/kaggle/input/tabulard/Test.csv').copy()
test_df['img_path']=test_df.Image_ID
test_df.drop(columns=['Image_ID'],inplace=True)
test_df

In [ ]:
# class CarTestDataset(object):
#     def __init__(self, df, IMG_DIR, transforms=None):
#         self.df = df
#         self.img_dir = IMG_DIR
#         self.image_ids = self.df['img_path'].unique().tolist()
#         self.transforms = transforms
#         print('hello')
#     def __len__(self):
#         return len(self.image_ids)
        
#     def __getitem__(self, idx):
#         image_id = self.image_ids[idx]
#         print(image_id)
#         records = self.df[self.df['img_path'] == image_id]
#         print(os.path.join(self.img_dir, image_id, '.png'))
#         image = cv2.imread(os.path.join(self.img_dir, image_id)+'.png', cv2.IMREAD_COLOR)
#         print('im',image)
#         image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
#         image /= 255.0
#         print(image)
#         # For test set, you don't have ground truth annotations, so return dummy targets
#         target = {}

#         if self.transforms:
#             sample = {
#                 'image': image,
#             }
#             sample = self.transforms(**sample)
#             image = sample['image']
            
#         return image.clone().detach(), image_id
# IMG_DIR_TEST = '/kaggle/input/imagest/test/test'
# def get_valid_transform():
#     return A.Compose([
#         ToTensorV2()
#     ], bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})

# # Example of how to use the test dataset with transformations
# test_transform = get_valid_transform()  # You can use valid transform or define a new transform
# test_dataset = CarTestDataset(test_df, IMG_DIR_TEST, transforms=test_transform)
# test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False)



In [ ]:
import os

class CarTestDataset(object):
    def __init__(self, df, IMG_DIR, transforms=None):
        self.df = df
        self.img_dir = IMG_DIR
        self.image_ids = self.df['img_path'].unique().tolist()
        self.transforms = transforms
        print('hello')
        
    def __len__(self):
        return len(self.image_ids)
        
    def __getitem__(self, idx):
        image_id = self.image_ids[idx]
#         print("image_id:", image_id)
#         print(self.img_dir, image_id)
        
        image_path = os.path.join(self.img_dir, image_id)+ '.png'
#         print("image_path:", image_path)
        
        if not os.path.exists(image_path):
            raise FileNotFoundError(f"Image file not found: {image_path}")
        
        image = cv2.imread(image_path, cv2.IMREAD_COLOR)
#         print("image:", image)
        
        if image is None:
            raise IOError(f"Unable to read image file: {image_path}")
        
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0
        
#         print("transformed image:", image)
        
        # For test set, you don't have ground truth annotations, so return dummy targets
        target = {}

        if self.transforms:
            sample = {
                'image': image,
            }
            sample = self.transforms(**sample)
            image = sample['image']
            
        return image.clone().detach(), image_id

# Example of how to use the test dataset with transformations
IMG_DIR_TEST = '/kaggle/input/imagest/test/test'

test_dataset = CarTestDataset(test_df, IMG_DIR_TEST)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False)


In [ ]:
# Example of how to use the test dataset with transformations
def get_valid_transform():
    return A.Compose([
        ToTensorV2()
    ])

test_transform = get_valid_transform()  # You can use valid transform or define a new transform
test_dataset = CarTestDataset(test_df, IMG_DIR_TEST, transforms=test_transform)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False)

# Visual inspection of a few samples
for i in range(5):
    image, image_id = test_dataset[i]
    print("Sample", i)
    print("Image ID:", image_id)
    print("Image shape:", image.shape)
    plt.imshow(image.permute(1, 2, 0))  # Convert from (C, H, W) to (H, W, C) for visualization
    plt.show()



In [ ]:
img,target = test_dataset[77]
model.eval()
with torch.no_grad():
    prediction = model([img.to('cuda')])[0]
    
nms_prediction = apply_nms(prediction, iou_thresh=0.7)

In [ ]:
print("PREDICTED BOXES")
fig = plt.figure(figsize=(14, 10))
sample = test_dataset[77]
img_int = torch.tensor(sample[0] * 255, dtype=torch.uint8)
plt.imshow(draw_bounding_boxes(img_int,
    nms_prediction['boxes'][nms_prediction['scores'] > 0.25], width=4
).permute(1, 2, 0))

In [ ]:
print("PREDICTED BOXES")
fig = plt.figure(figsize=(14, 10))
sample = test_dataset[77]
img_int = torch.tensor(sample[0] * 255, dtype=torch.uint8)
plt.imshow(draw_bounding_boxes(img_int,
    nms_prediction['boxes'][nms_prediction['scores'] > 0.5], width=4
).permute(1, 2, 0))

# Predicting All images in test

In [ ]:
nms_prediction['labels']

In [ ]:
reverse_label_mapper = {v: k for k, v in label_mapper.items()}
original_labels = [reverse_label_mapper[label.item()] for label in nms_prediction['labels']]

print(original_labels)

In [ ]:
import pandas as pd

# Create an empty list to store the results
results = []

# Iterate over all images in the test dataset
for idx in range(len(test_dataset)):
    img, img_id = test_dataset[idx]
    
    # Perform inference
    model.eval()
    with torch.no_grad():
        prediction = model([img.to('cuda')])[0]

    # Apply non-maximum suppression
    nms_prediction = apply_nms(prediction, iou_thresh=0.7)
    
    # Extract bounding box coordinates and scores
    boxes = nms_prediction['boxes'][nms_prediction['scores'] > 0.5]
    scores = nms_prediction['scores'][nms_prediction['scores'] > 0.5]
    labels = nms_prediction['labels'][nms_prediction['scores'] > 0.5]
    labels = [reverse_label_mapper[label.item()] for label in labels]
    # Convert image ID and bounding box coordinates to DataFrame
    for i in range(len(boxes)):
        result = {
            'image_id': img_id,
            'xmin': boxes[i][0].item(),
            'ymin': boxes[i][1].item(),
            'xmax': boxes[i][2].item(),
            'ymax': boxes[i][3].item(),
            'score': scores[i].item(),
            'class':labels[i]
        }
        results.append(result)

# Convert the list of dictionaries to a DataFrame
results_df = pd.DataFrame(results)

# Display the DataFrame
results_df

In [ ]:
results_df['Image_ID']=results_df['image_id']
results_df.drop(columns=['image_id'],inplace=True)
results_df['class']= results_df['class'].apply(lambda x: 'class_'+str(x))
results_df

In [ ]:
results_df['confidence']=results_df['score']
results_df=results_df.drop(columns=['score'])
results_df

In [ ]:
results_df.Image_ID.nunique()

In [ ]:
submission_file= pd.read_csv('/kaggle/input/tabulard/SampleSubmission.csv')
submission_file

In [ ]:
results_df.to_csv('teeth_detection_00_.csv',index=False)

# Upvote this this notebook was helpful! Thanks